In [80]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.inspection import permutation_importance

In [2]:
# Read data from .csv file
data = pd.read_csv('../data/heart_failure_records.csv')

There is no information on hyperparameter tuning of Random Forest classifier. The authors are aware of the concept, since they apply tuning to SVM and Multi-layer Perceptron, but mention Random Forest with other methods that do not require tuning, such as Logistic Regression. This implies that authors used default hyperparameters specified in the `randomForest` R package.

Based on this we can deduce that the hyperparameters are:
- number of trees: 500
- fraction of features sampled at each split: $\sqrt n$, where $n$ is the number of features
- fraction of observation sampled at each split: 1 
- maximum leaf nodes in each tree: unlimited
- minimum size of terminal nodes / minimum samples in a leaf: 1

It should be noted that while the default hyperparameter for `scikit-learn` implementation of Random Forest classifier are mostly the same, the most important hyperparameter, the number of trees grown, is different: 100. This can be only deduced from the code made available by the researchers and is not stated in the paper.

In [ ]:
roc_auc_scores = []
pr_auc_scores = []
accuracy_scores = []
f1_scores = []
tp_scores = []
tn_scores = []
mcc_scores = []

In [143]:

# Instantiate series for accumulating feature importance rankings
rf_mdi_importance = pd.Series(0, index=data.drop(columns=['DEATH_EVENT', 'time']).columns, name='mean_impurity_decrease_rank')
rf_perm_importance = pd.Series(0, index=data.drop(columns=['DEATH_EVENT', 'time']).columns, name='mean_accuracy_decrease_rank')

# We do not need to use the same seed as researchers, since random number generator implementations
# are different in R and NumPy, so the partitions will always be different.
# We set this only once, since we want different partitions in each run.
np.random.seed(12345)

# Both performance assessment and feature importance are averaged over 100 runs
for i in range(100):
    # Partition data into 80/20 training/test sets
    X_train, X_test, y_train, y_test = train_test_split(data.drop(columns=['DEATH_EVENT', 'time']), data['DEATH_EVENT'], test_size=0.2)

    # Instantiate, train, predict
    rf = RandomForestClassifier(n_estimators=500)
    rf.fit(X_train, y_train)
    y_pred = rf.predict(X_test)

    # Calculate performance assessment metrics
    roc_auc_scores.append(metrics.roc_auc_score(y_test, y_pred))
    y, x, _ = metrics.precision_recall_curve(y_test, y_pred)
    pr_auc_scores.append(metrics.auc(x, y))
    accuracy_scores.append(metrics.accuracy_score(y_test, y_pred))
    f1_scores.append(metrics.f1_score(y_test, y_pred))
    tp_scores.append(metrics.recall_score(y_test, y_pred))
    tn_scores.append(metrics.recall_score(y_test, y_pred, pos_label=0))
    mcc_scores.append(metrics.matthews_corrcoef(y_test, y_pred))

    # Partition data into 70/30 training/test - researchers used different split than in performance assessment
    X_train, X_test, y_train, y_test = train_test_split(data.drop(columns=['DEATH_EVENT', 'time']), data['DEATH_EVENT'], test_size=0.3)

    # Instantiate, train, predict using new partitions
    rf = RandomForestClassifier(n_estimators=500)
    rf.fit(X_train, y_train)

    # Calculate and rank feature importances
    rf_importance_1 = pd.Series(
        rf.feature_importances_,
        index=rf.feature_names_in_,
        name='mean_impurity_decrease'
    ).sort_values().rank(ascending=False)

    # Calculate permutation importance on training data, as in the paper
    result = permutation_importance(rf, X_train, y_train, n_repeats=5)
    rf_importance_2 = pd.Series(
        result.importances_mean,
        index=rf.feature_names_in_,
        name='mean_accuracy_decrease'
    ).sort_values().rank(ascending=False)

    # Accumulate rankings
    rf_mdi_importance += rf_importance_1
    rf_perm_importance += rf_importance_2

Researchers calculated all feature importance rankings on training data. We consider this a methodological mistake - with deep trees grown on such a small number of observations (`n=299`), the results are of little value, since they are likely driven by noise in the data, rather than underlying relationships. The goal of the paper is to use feature importance for selecting the top features, to be then used to make predictions on the test set with a more parsimonious model. With that goal in mind, it would be preferable in our view to split the data into training, validation and test set and calculate relevant importances on the validation set (e.g. permutation importance). Also, the authors state that the training set is 70% of all observations, which is incosistent to 80% stated in model performance assessment. 

Due to differences in random number generation, it is impossible to fully reproduce the figures in another language, however the researchers repeated importance calculation on 100 different splits, which should minimize the differences.

In [142]:
importances = pd.DataFrame(rf_mdi_importance).join(rf_perm_importance)

# Calculate aggregate ranking using Borda's method - i.e. sum individual ranks from multiple runs and rank the sums in ascending order
importances['rank_sum'] = importances['mean_accuracy_decrease_rank'] + importances['mean_impurity_decrease_rank']
importances['rank_agg'] = importances['rank_sum'].rank()

importances['rank_agg'].sort_values()

serum_creatinine             1.0
ejection_fraction            2.0
age                          3.0
serum_sodium                 4.0
creatinine_phosphokinase     5.0
platelets                    6.0
sex                          7.0
anaemia                      8.0
diabetes                     9.0
high_blood_pressure         10.0
smoking                     11.0
Name: rank_agg, dtype: float64

In [106]:


importances['rank_mdi_dec'] = importances['mean_impurity_decrease'].rank(ascending=False)
importances['rank_acc_dec'] = importances['mean_accuracy_decrease'].rank(ascending=False)
importances['rank_sum'] = importances['rank_mdi_dec'] + importances['rank_acc_dec']
importances['rank_borda'] = importances['rank_sum'].rank(method='min')
importances.sort_values('rank_borda')

,mean_impurity_decrease,mean_accuracy_decrease,rank_mdi_dec,rank_acc_dec,rank_sum,rank_borda
serum_creatinine,0.207998,0.046667,1.0,1.0,2.0,1.0
ejection_fraction,0.193029,0.035000,2.0,2.0,4.0,2.0
anaemia,0.023365,0.000000,7.0,3.0,10.0,3.0
serum_sodium,0.117418,-0.010000,4.0,6.0,10.0,3.0
age,0.155751,-0.023333,3.0,10.0,13.0,5.0
creatinine_phosphokinase,0.114392,-0.021667,5.0,8.5,13.5,6.0
smoking,0.017881,-0.001667,10.0,4.0,14.0,7.0
high_blood_pressure,0.022921,-0.016667,8.0,7.0,15.0,8.0
sex,0.017420,-0.003333,11.0,5.0,16.0,9.0
platelets,0.110927,-0.031667,6.0,11.0,17.0,10.0
